# Rewriting TransferFuncList

In [1]:
%load_ext autoreload
import sys
sys.path.append("../..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]

# SET THIS TO TRUE TO USE LATEX
matplotlib.rcParams['text.usetex'] = False

from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.low_energy.lowE_deposition import compute_fs
import main

from tqdm import tqdm_notebook as tqdm

np.set_printoptions(threshold=np.nan)

In [3]:
standard_tfs = False
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190120/'

string_fixed_xe = 'standard_xe_xHe_full_rs'
string_with_xHe_0 = 'xHe0'
string_with_xHe = 'full_xHe'

################# Standard xe Transfer Functions ##############################
# This is a TransferFuncList
highengphot_tflist_fixed_xe = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_fixed_xe+'.raw','rb'))
highengphot_tflist_fixed_xe = highengphot_tflist_fixed_xe[0]

################# xHe=0 Transfer Functions ####################################
highengphot_tflist_xHe_0 = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_with_xHe_0+'.raw', 'rb'))

xes_xHe_0 = 0.5 + 0.5*np.tanh(
    [-5., -4.1, -3.2, -2.3, -1.8, -1.4, -1, -.8, -0.65, -0.5, -.24, 0, .2, 0.4, .7, 1.3, 2.2, 3.1, 4]
)



################# Full xHe-Dependent Transfer Functions #######################

highengphot_tflist_arr_with_xHe = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_with_xHe+'.raw', 'rb'))

xes_with_xHe = 0.5 + 0.5*np.tanh(
    np.array(
        [-5.0, -2.3, -0.5, 0.0, 0.4, 2.2, 4.0]
    )
)

x_arr_with_xHe = np.array([[[xH, xHe] for xHe in xes_with_xHe] for xH in xes_with_xHe])


In [ ]:
# highengphot_tflist_arr_with_xHe
# highengphot_tflist_xHe_0
highengphot_tflist_fixed_xe

In [ ]:

highengphot_xHe_0_tflistarr = tflist.TransferFuncListArray(highengphot_tflist_xHe_0, xes_xHe_0) 
highengphot_full_xHe_tflistarr = tflist.TransferFuncListArray(highengphot_tflist_arr_with_xHe, x_arr_with_xHe)


In [ ]:


list_of_tflistarr = [
        highengphot_full_xHe_tflistarr, 
        highengphot_xHe_0_tflistarr, 
        highengphot_tflist_fixed_xe
    ]

a = tflist.TransferFuncInterp(list_of_tflistarr, rs_nodes=[30, 1600])

In [ ]:

a.get_tf(0.5, 0.5, 300)

In [ ]:
string_fixed_xe = 'standard_xe_xHe_full_rs'
string_with_xHe_0 = 'xHe0'
string_with_xHe = 'full_xHe'

highengdep_arr = pickle.load(
            open(
                direc+"highdeposited_60eV_complete_"
                +string_with_xHe+".raw", "rb"
            )
        )

In [ ]:
highengdep_arr.shape

In [4]:
%autoreload
direc_arr = ['/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190120/' for i in np.arange(3)]
xes = [x_arr_with_xHe, xes_xHe_0, None]
string_arr = [string_with_xHe, string_with_xHe_0, string_fixed_xe]

# direc_arr = ['/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190120/' for i in np.arange(2)]
# xes = [xes_xHe_0, None]
# string_arr = [string_with_xHe_0, string_fixed_xe]

CMB_subtracted = False

(highengphot_tf_interp,
     lowengphot_tf_interp,
     lowengelec_tf_interp,
     highengdep_interp,
     CMB_engloss_interp
    ) = main.load_trans_funcs(
        direc_arr, xes, 
        rs_nodes = np.array([40, 1600]),
#         rs_nodes = np.array([40, 1600]),
        string_arr=string_arr, inverted=False, CMB_subtracted=CMB_subtracted)


Loading transfer functions, set 0
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  0  complete.
Padding transfer functions...
High energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses...
Creating array objects...
Generating TransferFuncInterp object...
Loading transfer functions, set 1
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  1  complete.
Padding transfer functions...
High energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses...
Creating array objects...
Generating TransferFuncInterp object...
Loading transfer functions, set 2
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  2  complete.
Padding transfer fun

In [6]:
highengphot_tf_interp.get_tf(0.5, 0.9, 1600)